<a href="https://colab.research.google.com/github/gsilver321/project_voting/blob/main/voting_cleaning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import pandas as pd

! git clone https://github.com/gsilver321/project_voting

Cloning into 'project_voting'...
remote: Enumerating objects: 149, done.
remote: Counting objects: 100% (66/66), done.
remote: Compressing objects: 100% (38/38), done.
remote: Total 149 (delta 37), reused 46 (delta 28), pack-reused 83
Receiving objects: 100% (149/149), 124.63 MiB | 21.90 MiB/s, done.
Resolving deltas: 100% (66/66), done.
Updating files: 100% (68/68), done.


In [3]:
tsdf = pd.read_csv("project_voting/data/county_data/0002_ts_nominal_county.csv", low_memory=False, encoding="latin-1") # provided time series
tsdf = tsdf.loc[tsdf["STATE"] == "Virginia"] # limit time series to Virginia

tsdf2 = pd.read_csv("project_voting/data/county_data/nhgis0001_ts_nominal_county.csv", low_memory=False, encoding="latin-1") # downloaded time series with additional demographic info
tsdf2 = tsdf2.loc[tsdf2["STATE"] == "Virginia"]

df_2022 = pd.read_csv("project_voting/data/county_data/nhgis0002_ds261_2022_county.csv", low_memory=False, encoding="latin-1") # 2022 census data
df_2022 = df_2022.loc[df_2022["STATE"] == "Virginia"]

vdf = pd.read_csv('project_voting/data/voting_VA.csv')
votes = vdf.loc[:, ["year", "county_name", "party", "candidatevotes"]].groupby(["year", "county_name", "party"]).sum() # convert voting format to determine winners
votes

candidatevotes
year county_name party                      
2000 ACCOMACK    DEMOCRAT               5092
                 GREEN                   220
                 OTHER                   261
                 REPUBLICAN             6352
     ALBEMARLE   DEMOCRAT              16255
...                                      ...
2020 WYTHE       REPUBLICAN            11733
     YORK        DEMOCRAT              17683
                 LIBERTARIAN             680
                 OTHER                   187
                 REPUBLICAN            20241

[2596 rows x 1 columns]

In [4]:
win_arr = []

for year in votes.groupby(level=0): # loop through year
  for county in year[1].groupby(level=1): # loop through county
    county_votes = county[1]["candidatevotes"]
    for i, vote in enumerate(county_votes): # loop through votes
      # if current vote makes up the largest share of votes in that county for current year:
      if vote == county_votes.max():
        win_arr.append([year[0], county[0], county_votes.index[i][2]]) # make that party the winner

df_win = pd.DataFrame(win_arr, columns=["year", "county", "winner"])
df_win

,year,county,winner
0,2000,ACCOMACK,REPUBLICAN
1,2000,ALBEMARLE,REPUBLICAN
2,2000,ALEXANDRIA,DEMOCRAT
3,2000,ALLEGHANY,REPUBLICAN
4,2000,AMELIA,REPUBLICAN
...,...,...,...
773,2020,WILLIAMSBURG CITY,DEMOCRAT
774,2020,WINCHESTER CITY,DEMOCRAT
775,2020,WISE,REPUBLICAN
776,2020,WYTHE,REPUBLICAN


In [5]:
# only democrat and republican winners in 2020
winners_2020 = df_win.loc[df_win["year"] == 2020]
winners_2020["winner"].value_counts()

winner
REPUBLICAN    87
DEMOCRAT      46
Name: count, dtype: int64

In [6]:
winners_2020.loc[:, "winner"] = winners_2020["winner"].apply(lambda x: 1 if x == "DEMOCRAT" else 0) # encode party for later model
winners_2020.to_csv("df_train.csv", index=False)

In [59]:
df_avg = pd.DataFrame(tsdf["COUNTY"])

# Start demographic time series at year 2000 since that's when voting data starts:
# num of 25+ y.o. college-educated citizens in Virginia counties over time
df_edu = tsdf.loc[:, ["B69AC2000", "B69AC125", "B69AC195"]]
df_edu = df_edu.apply(pd.to_numeric)

# num of 16+ y.o. workers in Virginia counties over time
df_work = tsdf.loc[:, ["B84AA2000", "B84AA125", "B84AA195"]]
df_work = df_work.apply(pd.to_numeric)

# num of foreign-born citizens in Virginia counties over time
df_foreign = tsdf.loc[:, ["AT5AB2000", "AT5AB125", "AT5AB195"]]
df_foreign = df_foreign.apply(pd.to_numeric)

# num of female citizens in Virginia counties over time
df_female = tsdf.loc[:, ["AV1AB2000", "AV1AB125", "AV1AB195"]]
df_female = df_female.apply(pd.to_numeric)

# per capita income in Virginia counties over time
df_income = tsdf.loc[:, ["BD5AA2000", "BD5AA125", "BD5AA195"]]
df_income = df_income.apply(pd.to_numeric)

# num of citizens below poverty line in Virginia counties over time
df_poverty = tsdf.loc[:, ["CL6AA2000", "CL6AA125", "CL6AA195"]]
df_poverty = df_poverty.apply(pd.to_numeric)

# num of citizens in rural areas in Virginia counties over time
df_rural = tsdf2.loc[:, ["A57AD2000", "A57AD2010"]]
df_rural = df_rural.apply(pd.to_numeric)

# num of black or african american citizens in Virginia counties over time
df_black = tsdf2.loc[:, ["B18AB2000", "B18AB2010", "B18AB2020"]]
df_black = df_black.apply(pd.to_numeric)

# num of hispanic or latinx citizens in Virginia counties over time
df_latinx = tsdf2.loc[:, ["A35AA2000", "A35AA2010", "A35AA2020"]]
df_latinx = df_latinx.apply(pd.to_numeric)

# num of aapi citizens in Virginia counties over time
df_aapi = tsdf2.loc[:, ["B18AD2000", "B18AD2010", "B18AD2020"]]
df_aapi = df_aapi.apply(pd.to_numeric)

# num of never married citizens in Virginia counties over time
df_marry_male = tsdf2.loc[:, ["BL1AA2000"]]
df_marry_male = df_marry_male.apply(pd.to_numeric)
df_marry_male = df_marry_male.rename(columns={"BL1AA2000": "2000"})
df_marry_female = tsdf2.loc[:, ["BL1AG2000"]]
df_marry_female = df_marry_female.apply(pd.to_numeric)
df_marry_female = df_marry_female.rename(columns={"BL1AG2000": "2000"})
df_marry = df_marry_male.add(df_marry_female)

# num of 65+ y.o. citizens in Virginia counties over time
df_65 = tsdf2.loc[:, ["B57AP2000", "B57AP2010"]]
df_65 = df_65.apply(pd.to_numeric)
df_65 = df_65.rename(columns={"B57AP2000": "2000", "B57AP2010": "2010"})
df_75 = tsdf2.loc[:, ["B57AQ2000", "B57AQ2010"]]
df_75 = df_75.apply(pd.to_numeric)
df_75 = df_75.rename(columns={"B57AQ2000": "2000", "B57AQ2010": "2010"})
df_85 = tsdf2.loc[:, ["B57AR2000", "B57AR2010"]]
df_85 = df_85.apply(pd.to_numeric)
df_85 = df_85.rename(columns={"B57AR2000": "2000", "B57AR2010": "2010"})
df_over_65 = df_65.add(df_75).add(df_85)

# summarize time series for later model
df_avg["Avg College Degree"] = df_edu.mean(axis=1).values
df_avg["Avg Labor Force"] = df_work.mean(axis=1).values
df_avg["Avg Foreigners"] = df_foreign.mean(axis=1).values
df_avg["Avg Females"] = df_female.mean(axis=1).values
df_avg["Avg Income"] = df_income.mean(axis=1).values
df_avg["Avg Poverty"] = df_poverty.mean(axis=1).values
df_avg["Avg Rural"] = df_rural.mean(axis=1).values
df_avg["Avg Black or African American"] = df_black.mean(axis=1).values
df_avg["Avg Latinx"] = df_latinx.mean(axis=1).values
df_avg["Avg AAPI"] = df_aapi.mean(axis=1).values
df_avg["Avg Never Married"] = df_marry.mean(axis=1).values
df_avg["Avg 65+"] = df_over_65.mean(axis=1).values

df_avg

,COUNTY,Avg College Degree,Avg Labor Force,Avg Foreigners,Avg Females,Avg Income,Avg Poverty,Avg Rural,Avg Black or African American,Avg Latinx,Avg AAPI,Avg Never Married,Avg 65+
2845,Accomack County,4098.666667,16189.666667,2137.333333,17892.666667,21745.333333,6535.666667,32060.5,10020.666667,2780.666667,1809.333333,7241.0,6362.5
2846,Albemarle County,33353.000000,48481.333333,8938.333333,49732.000000,37162.666667,7565.000000,41390.5,9067.666667,5299.666667,7444.333333,15104.0,12022.0
2847,Alexandria city,63952.000000,92270.333333,36916.666667,73580.000000,51697.000000,12829.666667,0.0,30436.333333,23592.666667,20833.666667,42714.0,12205.5
2848,Alleghany County,1609.333333,6595.000000,204.666667,7535.666667,23551.000000,1833.666667,8813.5,586.333333,133.666667,93.333333,1625.0,2665.0
2849,Amelia County,1212.666667,6068.666667,165.333333,6184.333333,25031.333333,1198.333333,12045.0,2895.000000,268.666667,166.333333,1989.0,1756.5
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2977,Williamsburg city,3321.666667,6832.000000,969.000000,7400.333333,23749.333333,1759.666667,0.0,3089.333333,819.333333,977.666667,3341.0,1641.5
2978,Winchester city,5069.333333,13769.000000,2854.000000,13267.000000,26168.333333,4170.666667,0.0,2733.000000,3687.333333,2792.333333,6030.0,3553.5
2979,Wise County,3555.333333,15556.666667,423.333333,19589.333333,18154.666667,8299.000000,24113.0,1550.666667,405.000000,297.333333,6862.0,5726.5
2980,Wythe County,3139.000000,14134.333333,163.333333,14670.000000,23034.333333,3737.333333,21393.0,772.000000,264.000000,231.666667,4039.0,4787.5


In [60]:
print('----------- Pre-cleaning | demographic data null counts')
for v in df_avg.columns:
   print(v, sum(df_avg[v].isnull()))

print('\n', df_avg.head(), '\n')

for v in df_avg.columns: # loop through each var
  if v == "COUNTY": continue # skip categorical var
  df_avg[v] = df_avg[v].fillna(np.nanmedian(df_avg[v])) # fill nan with median for that column. Using median omits outliers

print('----------- Post-cleaning | demographic data null counts')
for v in df_avg.columns:
  print(v, sum(df_avg[v].isnull()))

print('\n', df_avg.head())

----------- Pre-cleaning | demographic data null counts
COUNTY 0
Avg College Degree 2
Avg Labor Force 2
Avg Foreigners 2
Avg Females 2
Avg Income 2
Avg Poverty 2
Avg Rural 2
Avg Black or African American 2
Avg Latinx 2
Avg AAPI 2
Avg Never Married 2
Avg 65+ 2

                 COUNTY  Avg College Degree  Avg Labor Force  Avg Foreigners  \
2845   Accomack County         4098.666667     16189.666667     2137.333333   
2846  Albemarle County        33353.000000     48481.333333     8938.333333   
2847   Alexandria city        63952.000000     92270.333333    36916.666667   
2848  Alleghany County         1609.333333      6595.000000      204.666667   
2849     Amelia County         1212.666667      6068.666667      165.333333   

       Avg Females    Avg Income   Avg Poverty  Avg Rural  \
2845  17892.666667  21745.333333   6535.666667    32060.5   
2846  49732.000000  37162.666667   7565.000000    41390.5   
2847  73580.000000  51697.000000  12829.666667        0.0   
2848   7535.666667 

In [61]:
df_win["encode_dem"] = df_win["winner"].apply(lambda x: 1 if x == "DEMOCRAT" else 0)
temp_df = df_win.loc[df_win["year"] != 2020] # exclude 2020 from probability since training on 2020. Add 2020 prob into test data later

temp_df = temp_df.loc[:, ["county", "encode_dem"]].groupby("county").mean() # excludes 2020 prob
temp_df2 = df_win.loc[:, ["county", "encode_dem"]].groupby("county").mean() # includes 2020 prob

missing_counties = []
for v in temp_df2.index:
  if v not in temp_df.index: # make counties same as 2020
    missing_counties.append(v)

index = list(temp_df.index) + missing_counties
temp_df = temp_df.reindex(index, fill_value=0.5) # give default 50/50 democrat chance for missing counties

for v in temp_df.index:
  if v.lower() not in [x.lower().replace("county", "").strip() for x in df_avg["COUNTY"]]:
    temp_df = temp_df.drop(v)

  if "city" not in v.lower():
    temp_df = temp_df.rename(index={v: v + " COUNTY"})

# remove counties not in map data (i.e. make counties same as ones in shapefile)
df_avg = df_avg[df_avg["COUNTY"] != "Bedford city"]
df_avg = df_avg[df_avg["COUNTY"] != "Nansemond County"]
df_avg = df_avg[df_avg["COUNTY"] != "South Boston City"]
df_avg = df_avg[df_avg["COUNTY"] != "Clifton Forge City"]

df_avg["Prob Dem"] = temp_df["encode_dem"].values
df_avg

,COUNTY,Avg College Degree,Avg Labor Force,Avg Foreigners,Avg Females,Avg Income,Avg Poverty,Avg Rural,Avg Black or African American,Avg Latinx,Avg AAPI,Avg Never Married,Avg 65+,Prob Dem
2845,Accomack County,4098.666667,16189.666667,2137.333333,17892.666667,21745.333333,6535.666667,32060.5,10020.666667,2780.666667,1809.333333,7241.0,6362.5,0.0
2846,Albemarle County,33353.000000,48481.333333,8938.333333,49732.000000,37162.666667,7565.000000,41390.5,9067.666667,5299.666667,7444.333333,15104.0,12022.0,0.8
2847,Alexandria city,63952.000000,92270.333333,36916.666667,73580.000000,51697.000000,12829.666667,0.0,30436.333333,23592.666667,20833.666667,42714.0,12205.5,0.0
2848,Alleghany County,1609.333333,6595.000000,204.666667,7535.666667,23551.000000,1833.666667,8813.5,586.333333,133.666667,93.333333,1625.0,2665.0,0.0
2849,Amelia County,1212.666667,6068.666667,165.333333,6184.333333,25031.333333,1198.333333,12045.0,2895.000000,268.666667,166.333333,1989.0,1756.5,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2977,Williamsburg city,3321.666667,6832.000000,969.000000,7400.333333,23749.333333,1759.666667,0.0,3089.333333,819.333333,977.666667,3341.0,1641.5,0.5
2978,Winchester city,5069.333333,13769.000000,2854.000000,13267.000000,26168.333333,4170.666667,0.0,2733.000000,3687.333333,2792.333333,6030.0,3553.5,0.5
2979,Wise County,3555.333333,15556.666667,423.333333,19589.333333,18154.666667,8299.000000,24113.0,1550.666667,405.000000,297.333333,6862.0,5726.5,0.5
2980,Wythe County,3139.000000,14134.333333,163.333333,14670.000000,23034.333333,3737.333333,21393.0,772.000000,264.000000,231.666667,4039.0,4787.5,0.5


In [62]:
df_avg["Prob Dem"].value_counts()

Prob Dem
0.0    70
0.5    38
1.0     8
0.4     6
0.6     5
0.8     3
0.2     3
Name: count, dtype: int64

In [53]:
df_avg.to_csv('demographic_info.csv', index=False)

In [76]:
df_test = pd.DataFrame(tsdf["COUNTY"])

# Extrapolate past data to get estimates for 2024 predictions:
edu = []
for r in df_edu.iterrows():
  fit = np.polyfit([2000, 2010, 2020], [r[1]["B69AC2000"], r[1]["B69AC125"], r[1]["B69AC195"]], 1)
  edu.append(np.poly1d(fit)(2024))

work = []
for r in df_work.iterrows():
  fit = np.polyfit([2000, 2010, 2020], [r[1]["B84AA2000"], r[1]["B84AA125"], r[1]["B84AA195"]], 1)
  work.append(np.poly1d(fit)(2024))

foreign = []
for r in df_foreign.iterrows():
  fit = np.polyfit([2000, 2010, 2020], [r[1]["AT5AB2000"], r[1]["AT5AB125"], r[1]["AT5AB195"]], 1)
  foreign.append(np.poly1d(fit)(2024))

female = []
for r in df_female.iterrows():
  fit = np.polyfit([2000, 2010, 2020], [r[1]["AV1AB2000"], r[1]["AV1AB125"], r[1]["AV1AB195"]], 1)
  female.append(np.poly1d(fit)(2024))

income = []
for r in df_income.iterrows():
  fit = np.polyfit([2000, 2010, 2020], [r[1]["BD5AA2000"], r[1]["BD5AA125"], r[1]["BD5AA195"]], 1)
  income.append(np.poly1d(fit)(2024))

poverty = []
for r in df_poverty.iterrows():
  fit = np.polyfit([2000, 2010, 2020], [r[1]["CL6AA2000"], r[1]["CL6AA125"], r[1]["CL6AA195"]], 1)
  poverty.append(np.poly1d(fit)(2024))

rural = []
for r in df_rural.iterrows():
  fit = np.polyfit([2000, 2010], [r[1]["A57AD2000"], r[1]["A57AD2010"]], 1)
  rural.append(np.poly1d(fit)(2024))

black = []
for r in df_black.iterrows():
  fit = np.polyfit([2000, 2010, 2020], [r[1]["B18AB2000"], r[1]["B18AB2010"], r[1]["B18AB2020"]], 1)
  black.append(np.poly1d(fit)(2024))

latinx = []
for r in df_latinx.iterrows():
  fit = np.polyfit([2000, 2010, 2020], [r[1]["A35AA2000"], r[1]["A35AA2010"], r[1]["A35AA2020"]], 1)
  latinx.append(np.poly1d(fit)(2024))

aapi = []
for r in df_aapi.iterrows():
  fit = np.polyfit([2000, 2010, 2020], [r[1]["B18AD2000"], r[1]["B18AD2010"], r[1]["B18AD2020"]], 1)
  aapi.append(np.poly1d(fit)(2024))

# need 2 points for extrapolation so redo df_marry even though some data older than 2000
df_marry_male = tsdf2.loc[:, ["BL1AA1990", "BL1AA2000"]]
df_marry_male = df_marry_male.apply(pd.to_numeric)
df_marry_male = df_marry_male.rename(columns={"BL1AA1990": "1990", "BL1AA2000": "2000"})
df_marry_female = tsdf2.loc[:, ["BL1AG1990", "BL1AG2000"]]
df_marry_female = df_marry_female.apply(pd.to_numeric)
df_marry_female = df_marry_female.rename(columns={"BL1AG1990": "1990", "BL1AG2000": "2000"})
df_marry = df_marry_male.add(df_marry_female)

marry = []
for r in df_marry.iterrows():
  fit = np.polyfit([2000, 2010], [r[1]["1990"], r[1]["2000"]], 1)
  marry.append(np.poly1d(fit)(2024))

over_65 = []
for r in df_over_65.iterrows():
  fit = np.polyfit([2000, 2010], [r[1]["2000"], r[1]["2010"]], 1)
  over_65.append(np.poly1d(fit)(2024))

df_test["College Degree"] = edu
df_test["Labor Force"] = work
df_test["Foreigners"] = foreign
df_test["Females"] = female
df_test["Income"] = income
df_test["Poverty"] = poverty
df_test["Rural"] = rural
df_test["Black or African American"] = black
df_test["Latinx"] = latinx
df_test["AAPI"] = aapi
df_test["Never Married"] = marry
df_test["65+"] = over_65

df_test

,COUNTY,College Degree,Labor Force,Foreigners,Females,Income,Poverty,Rural,Black or African American,Latinx,AAPI,Never Married,65+
2845,Accomack County,4846.266667,13599.666667,2767.333333,15809.466667,28541.633333,6082.766667,36253.8,7627.366667,3738.266667,2475.033333,9240.2,6261.8
2846,Albemarle County,43531.000000,58018.833333,12728.133333,60097.600000,48325.566667,10302.000000,53430.8,10679.766667,9796.466667,14034.833333,11975.0,20009.6
2847,Alexandria city,80402.700000,107522.633333,44151.866667,84325.000000,69220.800000,16209.966667,0.0,32527.933333,30935.666667,28092.666667,48822.2,14487.4
2848,Alleghany County,1920.133333,6795.200000,259.266667,8455.466667,28943.100000,3247.666667,7675.4,841.133333,225.366667,140.233333,972.6,5097.0
2849,Amelia County,1794.366667,6667.866667,213.633333,6600.833333,34012.333333,1497.233333,14496.0,2444.900000,502.466667,313.333333,2557.4,2678.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2977,Williamsburg city,4587.966667,6477.100000,1252.500000,8367.733333,31641.833333,2338.566667,0.0,5957.933333,1458.433333,1310.166667,-1391.0,2544.0
2978,Winchester city,7078.333333,15026.900000,4310.000000,14703.400000,33419.633333,5125.466667,0.0,3009.500000,6464.233333,4936.433333,7833.2,4019.0
2979,Wise County,4413.533333,14314.166667,583.633333,18002.433333,23214.966667,8277.300000,21749.4,2312.966667,517.000000,377.133333,7345.0,6252.8
2980,Wythe County,4301.700000,14324.033333,182.933333,14902.400000,29853.733333,4597.633333,23787.0,713.200000,402.600000,305.166667,3988.6,6400.6


In [77]:
print('----------- Pre-cleaning | test data null counts')
for v in df_test.columns:
   print(v, sum(df_test[v].isnull()))

print('\n', df_test.head(), '\n')

for v in df_test.columns: # loop through each var
  if v == "COUNTY": continue # skip categorical var
  df_test[v] = df_test[v].fillna(np.nanmedian(df_test[v])) # fill nan with median for that column. Using median omits outliers

print('----------- Post-cleaning | test data null counts')
for v in df_test.columns:
  print(v, sum(df_test[v].isnull()))

print('\n', df_test.head())

----------- Pre-cleaning | test data null counts
COUNTY 0
College Degree 4
Labor Force 4
Foreigners 4
Females 4
Income 4
Poverty 4
Rural 3
Black or African American 4
Latinx 4
AAPI 4
Never Married 2
65+ 3

                 COUNTY  College Degree    Labor Force    Foreigners  \
2845   Accomack County     4846.266667   13599.666667   2767.333333   
2846  Albemarle County    43531.000000   58018.833333  12728.133333   
2847   Alexandria city    80402.700000  107522.633333  44151.866667   
2848  Alleghany County     1920.133333    6795.200000    259.266667   
2849     Amelia County     1794.366667    6667.866667    213.633333   

           Females        Income       Poverty    Rural  \
2845  15809.466667  28541.633333   6082.766667  36253.8   
2846  60097.600000  48325.566667  10302.000000  53430.8   
2847  84325.000000  69220.800000  16209.966667      0.0   
2848   8455.466667  28943.100000   3247.666667   7675.4   
2849   6600.833333  34012.333333   1497.233333  14496.0   

      Black

In [79]:
for v in temp_df2.index:
  if v.lower() not in [x.lower().replace("county", "").strip() for x in df_test["COUNTY"]]:
    temp_df2 = temp_df2.drop(v)

  if "city" not in v.lower():
    temp_df2 = temp_df2.rename(index={v: v + " COUNTY"})

# remove counties not in map data (i.e. make counties same as ones in shapefile)
df_test = df_test[df_test["COUNTY"] != "Bedford city"]
df_test = df_test[df_test["COUNTY"] != "Nansemond County"]
df_test = df_test[df_test["COUNTY"] != "South Boston City"]
df_test = df_test[df_test["COUNTY"] != "Clifton Forge City"]

df_test["Prob Dem"] = temp_df2["encode_dem"].values
df_test

,COUNTY,College Degree,Labor Force,Foreigners,Females,Income,Poverty,Rural,Black or African American,Latinx,AAPI,Never Married,65+,Prob Dem
2845,Accomack County,4846.266667,13599.666667,2767.333333,15809.466667,28541.633333,6082.766667,36253.8,7627.366667,3738.266667,2475.033333,9240.2,6261.8,0.000000
2846,Albemarle County,43531.000000,58018.833333,12728.133333,60097.600000,48325.566667,10302.000000,53430.8,10679.766667,9796.466667,14034.833333,11975.0,20009.6,0.833333
2847,Alexandria city,80402.700000,107522.633333,44151.866667,84325.000000,69220.800000,16209.966667,0.0,32527.933333,30935.666667,28092.666667,48822.2,14487.4,1.000000
2848,Alleghany County,1920.133333,6795.200000,259.266667,8455.466667,28943.100000,3247.666667,7675.4,841.133333,225.366667,140.233333,972.6,5097.0,0.000000
2849,Amelia County,1794.366667,6667.866667,213.633333,6600.833333,34012.333333,1497.233333,14496.0,2444.900000,502.466667,313.333333,2557.4,2678.0,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2977,Williamsburg city,4587.966667,6477.100000,1252.500000,8367.733333,31641.833333,2338.566667,0.0,5957.933333,1458.433333,1310.166667,-1391.0,2544.0,1.000000
2978,Winchester city,7078.333333,15026.900000,4310.000000,14703.400000,33419.633333,5125.466667,0.0,3009.500000,6464.233333,4936.433333,7833.2,4019.0,1.000000
2979,Wise County,4413.533333,14314.166667,583.633333,18002.433333,23214.966667,8277.300000,21749.4,2312.966667,517.000000,377.133333,7345.0,6252.8,0.000000
2980,Wythe County,4301.700000,14324.033333,182.933333,14902.400000,29853.733333,4597.633333,23787.0,713.200000,402.600000,305.166667,3988.6,6400.6,0.000000


In [81]:
df_test["Prob Dem"].value_counts() # different that prior data which ommitted results of 2020 election

Prob Dem
0.000000    76
1.000000    36
0.166667     6
0.333333     5
0.833333     4
0.500000     3
0.666667     3
Name: count, dtype: int64

In [82]:
df_avg.to_csv('df_test.csv', index=False)